In [ ]:
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np


def visu_rectangle(image_path,image_name,bbox_all,label,outpath):
    row,col = np.shape(bbox_all)
    im_file = os.path.join(image_path, image_name)
    im = Image.open(im_file)  
    bbox = np.array([0,0,0,0])
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(im, aspect='equal')

    colors = {'DET_FP' : 'red',
               'GT_FN'  : 'yellow',
               'DET_TP' : 'green',
               'GT_TP'  : 'green',}
    
    
    for r in range(0,row) :
        x_scale = 4.5
        y_scale = 4.53
        bbox[0] = bbox_all[r][0]*x_scale
        bbox[1] = bbox_all[r][1]*y_scale
        bbox[2] = bbox_all[r][2]*x_scale
        bbox[3] = bbox_all[r][3]*y_scale
        
        if row == 1:
            idx = label
        else: 
            idx = label[r]

        ax.add_patch(plt.Rectangle((bbox[0], bbox[1]),
            bbox[2] - bbox[0],
            bbox[3] - bbox[1], fill=False,
            edgecolor=colors[idx], linewidth=3.5))

        ax.text(bbox[0], bbox[1] - 2,
           '{:s} {:s}'.format('person', idx),
           bbox=dict(facecolor='blue', alpha=0.5),
           fontsize=14, color='white')    
    
    outFile =  outpath + image_name
    fig.savefig(outFile)
    plt.close(fig)

In [ ]:
# visualize reports TP/FP,FN for RESINDOOR (!! rescaling to original resolution)
import pandas as pd

CSV_file = 'r:/Projects/2017_ObjectClassification/data/detection_reports/ResiIndoor_annotations.xgtf_ResiIndoor_SightHound.xgtf_person_1-9480_0.3.csv'
outpath = "c:/Users/E411208/Documents/Python Scripts/data/Report/SightHound/" 

#CSV_file = 'r:/Projects/2017_ObjectClassification/data/detection_reports/ResiIndoor_annotations.xgtf_ResiIndoor_YOLO_small_threshold_005_tracked3.xgtf_person_1-9480_0.3.csv'
#outpath = "c:/Users/E411208/Documents/Python Scripts/data/Report/YOLO_small/" 

#CSV_file = 'r:/Projects/2017_ObjectClassification/data/detection_reports/ResiIndoor_annotations.xgtf_ResiIndoor_annotation_plus_FasterRCNN_output_80.xgtf_person_1-9480_0.3.csv'
#outpath = "c:/Users/E411208/Documents/Python Scripts/data/Report/Ann_plus_faster_0_8/" 

image_path = 'C:/Users/E411208/Documents/Python Scripts/data/ResiIndoor/Img/'


df = pd.read_csv(CSV_file)
df.rename(columns = {' DET_FP||GT_FN||DET_TP||GT_TP [':'Output'}, inplace=True)

# sort by frame
df = df.set_index('frame')
df = df.sort_index()
#print(df)
#DET_FP = df[df['Output'] == "DET_FP"]
#print(DET_FP)
image_name_prev = ''

counter = 1

for i, row in enumerate(df.values):
    print(row)
    id,x,y,u,v,label,matchID = row
    image_name = "output_{:05d}.jpg".format(df.index[i])
    #print(image_name)
    
    #if df.index[i] == 1066:
    if image_name_prev == image_name:
       bbox = np.append(bbox, [x,y,u,v])
       labels = np.append(labels,label)
       counter = counter+1
    else :
       bbox =  np.array([x,y,u,v])
       labels =  label
       counter = 1 

    bbox = bbox.reshape(counter,4)
    print(image_name)
    print(bbox)
    #print(labels)
    visu_rectangle(image_path,image_name,bbox,labels,outpath)
    image_name_prev = image_name   
    

In [1]:
# NON-MAX-SUPPRESION
# import the necessary packages
import numpy as np
 
# Malisiewicz et al.
def non_max_suppression_fast(boxes,score, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
 
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
 
    # initialize the list of picked indexes	
    pick = []
 
    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
 
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    #idxs = np.argsort(y2)
    idxs = np.argsort(area)
 
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
 
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
 
        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
 
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
 
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
 
    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick].astype("int"), score[pick].astype("float")

In [2]:
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np


def visu_rectangle(image_path,image_name,bbox_all,score,outpath):
    row,col = np.shape(bbox_all)
    im_file = os.path.join(image_path, image_name)
    im = Image.open(im_file)  
    bbox = np.array([0,0,0,0])
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(im, aspect='equal')

    for r in range(0,row) :
        if score[r] > 0.2 :
            x_scale = 1#4.5
            y_scale = 1#4.53
            bbox[0] = bbox_all[r][0]*x_scale
            bbox[1] = bbox_all[r][1]*y_scale
            bbox[2] = bbox_all[r][2]*x_scale
            bbox[3] = bbox_all[r][3]*y_scale

            ax.add_patch(plt.Rectangle((bbox[0], bbox[1]),
                bbox[2] - bbox[0],
                bbox[3] - bbox[1], fill=False,
                edgecolor='red', linewidth=3.5))

            ax.text(bbox[0], bbox[1] - 2,
               '{:s} {:.3f}'.format('person', score[r]),
               bbox=dict(facecolor='blue', alpha=0.5),
               fontsize=14, color='white')    
    
    outFile =  outpath + image_name
    fig.savefig(outFile)
    plt.close(fig)


In [3]:
# visualize results in all subfolders
import pandas as pd
# MAIN
#process_path = 'z:/OC_RESINET/process/NestCam-BB/'
process_path = 'z:/OC_RESINET/process/ResiIndoor05-Pond5/'
#process_path = 'z:/OC_RESINET/ResiIndoor/'
USE_NMS = False 

#for subdir in os.listdir(process_path):
for i in range(0,1):
    #subdir = 'indoor_002/'
    #subdir = 'indoor_door_002/'
    subdir = 'front_door_001/'
    #subdir = 'man_pet_005'
    image_path = os.path.join(process_path, subdir)
    if os.path.isdir(image_path):
        print(image_path)
        
        #CSV_file = image_path + '/FasterRCNN_output.csv'
        #outpath =  image_path + '/_out-Faster/'
        
        #CSV_file = image_path + '/YOLOv2_0.1.csv'
        #outpath =  image_path + '/_out-Yolo2/'


        #CSV_file = image_path + '/YOLO_small_threshold_002.csv'
        #outpath =  image_path + '/_out-Yolo-small-002/'
        
        #CSV_file = image_path + '/YOLO_small_threshold_005.csv'
        #outpath =  image_path + '/_out-Yolo-small/'
        
        #CSV_file = image_path + '/YOLO_small_threshold_005_nonoverlapped.csv'
        #outpath =  image_path + '/_out-Yolo-small-clean/'

        #CSV_file = image_path + '/HOGdetector_output.csv'
        #outpath =  image_path + '/_out-HOGdetector/'

        #CSV_file = image_path + '/BGS_GMG_output.csv'
        #outpath =  image_path + '/_out-BGS_GMG/'

        #CSV_file = image_path + '/BGS_MOG_output.csv'
        #outpath =  image_path + '/_out-BGS_MOG/'

        #CSV_file = image_path + '/BGS_EIG_output.csv'
        #outpath =  image_path + '/_out-BGS_EIG/'


        #CSV_file = image_path + '/SightHound_output_vis.csv'
        #outpath =  image_path + '/_out-SightHound/'
        
        #CSV_file = image_path + 'bgs_output_small.csv '
        #outpath =  image_path + '_out_bgs/'
        if USE_NMS:
            outpath = outpath + '_clean'

        # create output folder
        if not os.path.exists(outpath):
            os.makedirs(outpath)

        image_name_prev = 'output_00001.jpg'
        bbox = np.array([0,0,0,0])
        conf = np.array([0])
        counter = 1

        try:
            df = pd.read_csv(CSV_file)
            df = df.T
            df = df.reset_index()
            df = df.T

            df.rename(columns={0:'Filename'}, inplace=True)
            df.rename(columns={1:'x1'}, inplace=True)
            df.rename(columns={2:'y1'}, inplace=True)
            df.rename(columns={3:'x2'}, inplace=True)
            df.rename(columns={4:'y2'}, inplace=True)
            df.rename(columns={5:'score'}, inplace=True)
            df.rename(columns={6:'u1'}, inplace=True)
            df.rename(columns={7:'v1'}, inplace=True)
            df.rename(columns={8:'u2'}, inplace=True)
            df.rename(columns={9:'v2'}, inplace=True)    

            df = df.set_index('Filename')
            df = df.sort_index()
            
            #print(df)

            for i, row in enumerate(df.values):
                image_name = df.index[i]
                x,y,u,v,s,a,b,c,d = row

                if image_name_prev == image_name:
                    #print('same_image')
                    plot_image = False
                else :
                    if counter > 1 :
                        if USE_NMS:
                            bbox,conf = non_max_suppression_fast(bbox,conf, 0.40)
                        #print(image_name,counter,bbox)
                        visu_rectangle(image_path,image_name_prev,bbox,conf,outpath)
                    bbox = np.array([0,0,0,0])
                    conf = np.array([0])
                    counter = 1 
                    plot_image = True
                
                
                bbox = np.append(bbox, [float(x),float(y),float(u),float(v)])
                conf = np.append(conf,float(s))
                counter = counter+1
                

                bbox = bbox.reshape(counter,4)
                conf = np.array(conf,'float')
                if plot_image is True:
                    visu_rectangle(image_path,image_name,bbox,conf,outpath)
                image_name_prev = image_name   

        except :
            print('error in loading results')

print('DONE')

z:/OC_RESINET/process/ResiIndoor05-Pond5/front_door_001/
error in loading results
DONE


In [7]:
# visualize results in all subfolders for TRACKER RESULTS
import pandas as pd
# MAIN
#process_path = 'z:/OC_RESINET/process/NestCam-BB/'
process_path = 'z:/OC_RESINET/process/ResiIndoor05-Pond5/'
#process_path = 'z:/OC_RESINET/ResiIndoor/'
USE_NMS = True 

#for subdir in os.listdir(process_path):
for i in range(0,1):
    #subdir = 'indoor_002/'
    #subdir = 'indoor_door_002/'
    subdir = 'front_door_001/'
    #subdir = 'man_pet_005'
    image_path = os.path.join(process_path, subdir)
    if os.path.isdir(image_path):
        print(image_path)
        
        CSV_file = image_path + '/YOLOv2_0.1_elim94SORTtracked_output_30.csv'
        outpath =  image_path + '/_out-Yolo2_trk'
        if USE_NMS:
            outpath = outpath + '_clean/'
        else :
            outpath = outpath + '/'

        # create output folder
        if not os.path.exists(outpath):
            os.makedirs(outpath)

        image_name_prev = 'output_00001.jpg'
        bbox = np.array([0,0,0,0])
        conf = np.array([0])
        counter = 1

        try:
            df = pd.read_csv(CSV_file)
            df = df.T
            df = df.reset_index()
            df = df.T

            df.rename(columns={0:'Filename'}, inplace=True)
            df.rename(columns={1:'ID'}, inplace=True)
            df.rename(columns={2:'x1'}, inplace=True)
            df.rename(columns={3:'y1'}, inplace=True)
            df.rename(columns={4:'x2'}, inplace=True)
            df.rename(columns={5:'y2'}, inplace=True)
            df.rename(columns={6:'score'}, inplace=True)

            df = df.set_index('Filename')
            df = df.sort_index()
            
            #remove lines with ID that has mean score + std < Threshold 
            
            print(df)

            for i, row in enumerate(df.values):
                image_name = df.index[i]
                id,x,y,u,v,score = row

                if image_name_prev == image_name:
                    #print('same_image')
                    plot_image = False
                else :
                    if counter > 1 :
                        if USE_NMS:
                            bbox,conf = non_max_suppression_fast(bbox,conf, 0.40)
                        #print(image_name,counter,bbox)
                        visu_rectangle(image_path,image_name_prev,bbox,conf,outpath)
                    bbox = np.array([0,0,0,0])
                    conf = np.array([0])
                    counter = 1 
                    plot_image = True
                
                
                bbox = np.append(bbox, [float(x),float(y),float(u),float(v)])
                conf = np.append(conf,float(score))
                counter = counter+1
                

                bbox = bbox.reshape(counter,4)
                conf = np.array(conf,'float')
                if plot_image is True:
                    visu_rectangle(image_path,image_name,bbox,conf,outpath)
                image_name_prev = image_name               
            
        except :
            print('error in loading results')            
            
print('DONE')            

z:/OC_RESINET/process/ResiIndoor05-Pond5/front_door_001/
                 ID   x1   y1   x2   y2       score
Filename                                           
output_00050.jpg  1  424  398  482  538  0.21195756
output_00051.jpg  1  423  406  484  554    0.211958
output_00052.jpg  1  422  393  490  540    0.221065
output_00053.jpg  1  419  394  501  520    0.204774
output_00054.jpg  1  418  394  504  525    0.204774
output_00055.jpg  1  412  385  520  512    0.250012
output_00056.jpg  1  412  382  524  514    0.250012
output_00057.jpg  1  400  395  542  536    0.334352
output_00058.jpg  1  391  395  548  540    0.303244
output_00059.jpg  1  390  394  553  545    0.303244
output_00060.jpg  1  389  393  557  549    0.303244
output_00061.jpg  1  388  393  562  554    0.303244
output_00062.jpg  1  347  345  584  540    0.246254
output_00063.jpg  1  344  341  588  542    0.246254
output_00064.jpg  1  341  337  593  544    0.246254
output_00065.jpg  1  338  333  597  546    0.246254
output_

In [ ]:
%run eliminateoverlaps.py

In [ ]:
# copy empty frames to folder
from shutil import copyfile
#for subdir in os.listdir(process_path):
process_path = 'z:/OC_RESINET/process/ResiIndoor05-Pond5/'

for i in range(0,1):
    #subdir = 'indoor_060/.'
    #subdir = 'indoor_door_004/.'
    subdir = 'front_door_014/.'
    image_path = os.path.join(process_path, subdir)
    if os.path.isdir(image_path):
        image_name = os.listdir(image_path)
        for img in image_name:
            if img.endswith('jpg'):
                src = image_path[:-2]  + img
                #dst = image_path[:-2]  + '/_out-Faster_clean/' 
                #dst = image_path[:-2]  + '/_out-SightHound/' 
                dst = image_path[:-2]  + '/_out-HOGdetector/'
                #dst = image_path[:-2]  + '/_out-Yolo-small_clean/' 
                
                if os.path.isfile(dst + img) is False:
                    bbox = np.array([0,0,0,0])
                    conf = np.array([0])
                    bbox = np.append(bbox, [0,0,0,0])
                    conf = np.append(conf,0)
                    counter = 2
                    bbox = bbox.reshape(counter,4)
                    conf = np.array(conf,'float')
                    visu_rectangle(image_path,img,bbox,conf,dst)
                    
print('DONE')

In [ ]:
# restore croped image and visualize detection results
import pandas as pd
# MAIN
#process_path = 'z:/OC_RESINET/process/NestCam-BB/'
#process_path = 'z:/OC_RESINET/process/ResiIndoor05-Pond5/'
process_path = 'z:/OC_RESINET/ResiIndoor/'
for subdir in os.listdir(process_path):
    image_path = os.path.join(process_path, subdir)
    if os.path.isdir(image_path) and (subdir[0] != '_'):
        #print(image_path)
        #CSV_file = image_path + '/FasterRCNN_output.csv'
        #outpath =  image_path + '/_out-Faster/'

        CSV_file = image_path + '/YOLO_small_threshold_005.csv'
        outpath =  image_path + '/_out-Yolo-small/'

        CSV_file = image_path +'/HOGdetector_output.csv'
        outpath =  image_path + '/_out-HOGdetector/'
        # create output folder
        if not os.path.exists(outpath):
            os.makedirs(outpath)

        image_name_prev = ''
        bbox = np.array([0,0,0,0])
        conf = np.array([0])
        counter = 1
        
        try:
            df = pd.read_csv(CSV_file)
            df = df.T
            df = df.reset_index()
            df = df.T

            df.rename(columns={0:'Filename'}, inplace=True)
            df.rename(columns={1:'x1'}, inplace=True)
            df.rename(columns={2:'y1'}, inplace=True)
            df.rename(columns={3:'x2'}, inplace=True)
            df.rename(columns={4:'y2'}, inplace=True)
            df.rename(columns={5:'score'}, inplace=True)
            df.rename(columns={6:'u1'}, inplace=True)
            df.rename(columns={7:'v1'}, inplace=True)
            df.rename(columns={8:'u2'}, inplace=True)
            df.rename(columns={9:'v2'}, inplace=True)    

            #df['Filename'] = df['Filename'].map(lambda x: x[4:])
            
            df = df.set_index('Filename')
            df = df.sort_index()

            #print(df)
           
            for i, row in enumerate(df.values):
                image_name = df.index[i]
                image_name = (image_name[-16:])
                x,y,u,v,s,a,b,c,d = row

                if image_name_prev == image_name:
                   print('same_image')                 
                else :
                    bbox = np.array([0,0,0,0])
                    conf = np.array([0])
                    counter = 1 

                print(image_name)
                bbox = np.append(bbox, [float(x)+float(a),float(y)+float(b),float(u)+float(a),float(v)+float(b)])
                conf = np.append(conf,s)
                counter = counter+1
                bbox = bbox.reshape(counter,4)
                conf = np.array(conf,'float')
                visu_rectangle(image_path,image_name,bbox,conf,outpath)
                image_name_prev = image_name   
            
        except :
            print('empty file')